In [1]:
import pandas as pd
import itertools
import os

In [2]:
pd.options.display.float_format = "{:,.3f}".format

### Globals, Helper Functions, etc

In [3]:
MEASURE_NAME_DICT = {
    "AlignedCosineSimilarity": "AlignCos",
    "CKA": "CKA",
    "ConcentricityDifference": "ConcDiff",
    "DistanceCorrelation": "DistCorr",
    "EigenspaceOverlapScore": "EOS",
    "GeometryScore": "GS",
    "Gulp": "GULP",
    "HardCorrelationMatch": "HardCorr",
    "IMDScore": "IMD",
    "JaccardSimilarity": "Jaccard",
    "LinearRegression": "LinReg",
    "MagnitudeDifference": "MagDiff",
    "OrthogonalAngularShapeMetricCentered": "ShapeMet",
    "OrthogonalProcrustesCenteredAndNormalized": "OrthProc",
    "PWCCA": "PWCCA",
    "PermutationProcrustes": "PermProc",
    "ProcrustesSizeAndShapeDistance": "ProcDist",
    "RSA": "RSA",
    "RSMNormDifference": "RSMDiff",
    "RankSimilarity": "RankSim",
    "SVCCA": "SVCCA",
    "SecondOrderCosineSimilarity": "2nd-Cos",
    "SoftCorrelationMatch": "SoftCorr",
    "UniformityDifference": "UnifDiff",
}

PIVOT_COL_DICT = {
    "cora": "Cora",
    "flickr": "Flickr",
    "ogbn-arxiv": "OGBN-Arxiv",
    "GraphSAGE": "SAGE",
    "violation_rate": "Violation Rate",
    "correlation": "Spearman Correlation"
}

COLUMN_NAME_DICT = {
    "similarity_measure": "Similarity Measure", 
    "quality_measure": "Measure",
    "functional_similarity_measure": "Measure",
    "architecture": "Model",
    "representation_dataset": "Dataset"
}

LATEX_FORMAT_DICT = {
    "float_format": "%.2f", 
    "column_format": "l||rrr|rrr|rrr||rrr|rrr|rrr",
    "multicolumn_format": "c",
    "index_names": False,
}

In [4]:
EXPERIMENT_RESULTS_PATH = 'C:/Users/Tobias/Eigene Dokumente/Research/similaritybench/experiments/results/full_csvs'
COMPARISON_TYPE_DICT = {
    "label_test": "group_separation",
    "layer_test": "monotonicity",
    "augmentation_test": "group_separation",
    "shortcut_test": "group_separation"
}

EXPERIMENTS = list(COMPARISON_TYPE_DICT.keys())
DATASETS = ["cora", "flickr", "ogbn-arxiv"]
ARCHITECTURES = ["GCN", "GraphSAGE", "GAT"]

def FULL_DF_FILE_NAME(experiment, comparison_type, dataset, groups=5):
    if groups < 5:
        return f"{experiment}_{comparison_type}_{dataset}_{groups}groups_full.csv"
    return f"{experiment}_{comparison_type}_{dataset}_full.csv"

In [5]:
def get_pivot_table(experiment, dataset):
    path = os.path.join(EXPERIMENT_RESULTS_PATH, FULL_DF_FILE_NAME(experiment, COMPARISON_TYPE_DICT[experiment], dataset))
    
    df = pd.read_csv(path)
    data = df.loc[:, ["similarity_measure", "quality_measure", "value", "architecture", "representation_dataset"]]
    return data.pivot(index="similarity_measure", columns=["representation_dataset", "architecture", "quality_measure"], values="value")

In [6]:
def get_agg_pivot_table(experiment, datasets, groups=5):
    dfs = []
    for dataset in datasets:
        path = os.path.join(EXPERIMENT_RESULTS_PATH, FULL_DF_FILE_NAME(experiment, COMPARISON_TYPE_DICT[experiment], dataset, groups))
        df = pd.read_csv(path)
        data = df.loc[:, ["similarity_measure", "quality_measure", "value", "architecture", "representation_dataset"]].dropna()
   
        data = data.rename(COLUMN_NAME_DICT, axis="columns")
        dfs.append(data.iloc[:])
    df_cc = pd.concat(dfs, axis=0)
    df_res = df_cc.pivot_table(index="Similarity Measure", columns=["Measure", "Dataset", "Model"], values="value", aggfunc="mean")
    df_res = df_res.rename(MEASURE_NAME_DICT, axis="index")
    return df_res.rename(PIVOT_COL_DICT, axis="columns")

### Results for Group Separation Experiments

#### Random Label Test Results

In [7]:
get_agg_pivot_table("label_test", ["cora", "flickr", "ogbn-arxiv"])

Measure            AUPRC                                                  \
Dataset             Cora             Flickr             OGBN-Arxiv         
Model                GAT   GCN  SAGE    GAT   GCN  SAGE        GAT   GCN   
Similarity Measure                                                         
AlignCos           0.278 0.289 0.287  0.183 0.513 0.275      0.459 0.467   
CKA                0.274 0.273 0.274  0.141 0.445 0.415      0.728 0.846   
ConcDiff           0.212 0.131 0.198  0.156 0.218 0.235      0.505 0.518   
DistCorr           0.274 0.280 0.287  0.116 0.491 0.308      0.722 0.804   
EOS                0.132 0.160 0.168  0.123 0.268 0.284      0.222 0.170   
GULP               0.133 0.160 0.168  0.118 0.106 0.285      0.197 0.194   
HardCorr           0.270 0.272 0.273  0.191 0.389 0.332      0.334 0.520   
Jaccard            0.274 0.273 0.273  0.131 0.326 0.296      0.385 0.565   
LinReg             0.286 0.282 0.284  0.125 0.130 0.326      0.297 0.279   
MagDiff            0.185 0.119 0.134  0.110 0.410 0.273      0.172 0.270   
ShapeMet           0.273 0.273 0.276  0.143 0.537 0.313      0.454 0.494   
OrthProc           0.273 0.273 0.276  0.143 0.537 0.313      0.454 0.494   
PermProc           0.277 0.288 0.245  0.115 0.474 0.427      0.524 0.225   
ProcDist           0.280 0.286 0.290  0.115 0.456 0.394      0.624 0.425   
RSA                0.274 0.291 0.275  0.201 0.404 0.278      0.319 0.527   
RSMDiff            0.379 0.309 0.307    NaN   NaN   NaN        NaN   NaN   
RankSim            0.299 0.308 0.270  0.167 0.305 0.318      0.410 0.643   
SVCCA              0.272 0.223 0.193  0.137 0.237 0.408      0.658 0.869   
2nd-Cos            0.346 0.338 0.370  0.216 0.326 0.279      0.596 0.745   
SoftCorr           0.272 0.272 0.274  0.174 0.298 0.361      0.332 0.491   
UnifDiff           0.165 0.158 0.292  0.207 0.341 0.419      0.205 0.324   

Measure                  Violation Rate                                 \
Dataset                            Cora             Flickr               
Model               SAGE            GAT   GCN  SAGE    GAT   GCN  SAGE   
Similarity Measure                                                       
AlignCos           0.464          0.427 0.414 0.389  0.399 0.145 0.492   
CKA                0.800          0.489 0.477 0.473  0.430 0.142 0.222   
ConcDiff           0.543          0.360 0.495 0.328  0.430 0.347 0.245   
DistCorr           0.822          0.479 0.427 0.397  0.454 0.123 0.354   
EOS                0.276          0.456 0.457 0.480  0.471 0.487 0.426   
GULP               0.277          0.450 0.448 0.478  0.445 0.494 0.428   
HardCorr           0.515          0.500 0.483 0.474  0.374 0.148 0.288   
Jaccard            0.283          0.462 0.475 0.490  0.439 0.250 0.371   
LinReg             0.367          0.399 0.411 0.429  0.482 0.530 0.300   
MagDiff            0.199          0.359 0.502 0.400  0.509 0.193 0.214   
ShapeMet           0.449          0.481 0.485 0.466  0.435 0.105 0.331   
OrthProc           0.449          0.481 0.485 0.466  0.435 0.105 0.331   
PermProc           0.257          0.413 0.377 0.355  0.500 0.165 0.167   
ProcDist           0.480          0.414 0.371 0.366  0.494 0.114 0.221   
RSA                0.284          0.482 0.386 0.456  0.393 0.173 0.454   
RSMDiff              NaN          0.249 0.357 0.257    NaN   NaN   NaN   
RankSim            0.354          0.393 0.369 0.487  0.443 0.319 0.399   
SVCCA              0.723          0.429 0.395 0.436  0.445 0.294 0.201   
2nd-Cos            0.710          0.221 0.245 0.218  0.389 0.255 0.473   
SoftCorr           0.531          0.494 0.485 0.477  0.390 0.219 0.287   
UnifDiff           0.244          0.421 0.397 0.286  0.331 0.190 0.241   

Measure                                    
Dataset            OGBN-Arxiv              
Model                     GAT   GCN  SAGE  
Similarity Measure                         
AlignCos                0.196 0.198 0.348  
CKA                     0.05

In [8]:
label_test_df = get_agg_pivot_table("label_test", ["cora", "flickr", "ogbn-arxiv"], groups=3)
label_test_df.to_latex("graphs_results_label_test.tex", 
                       **LATEX_FORMAT_DICT
                      )
label_test_df

Measure            AUPRC                                                  \
Dataset             Cora             Flickr             OGBN-Arxiv         
Model                GAT   GCN  SAGE    GAT   GCN  SAGE        GAT   GCN   
Similarity Measure                                                         
AlignCos           0.433 0.457 0.479  0.286 0.839 0.423      0.931 0.981   
CKA                0.424 0.427 0.424  0.270 0.733 0.657      1.000 1.000   
ConcDiff           0.394 0.203 0.391  0.208 0.366 0.570      0.997 0.964   
DistCorr           0.424 0.431 0.444  0.217 0.860 0.427      1.000 1.000   
EOS                0.284 0.295 0.259  0.219 0.413 0.424      0.340 0.253   
GULP               0.285 0.276 0.260  0.227 0.192 0.424      0.303 0.302   
HardCorr           0.424 0.425 0.424  0.332 0.771 0.457      0.544 0.828   
Jaccard            0.427 0.425 0.424  0.292 0.564 0.430      0.778 0.827   
LinReg             0.456 0.492 0.431  0.230 0.223 0.449      0.467 0.444   
MagDiff            0.370 0.199 0.243  0.185 0.661 0.718      0.343 0.551   
ShapeMet           0.424 0.426 0.424  0.275 0.882 0.434      0.767 0.826   
OrthProc           0.424 0.426 0.424  0.275 0.882 0.434      0.767 0.826   
PermProc           0.435 0.443 0.386  0.192 0.772 0.904      0.931 0.678   
ProcDist           0.433 0.452 0.451  0.190 0.785 0.625      1.000 0.915   
RSA                0.425 0.457 0.435  0.334 0.737 0.424      0.490 0.961   
RankSim            0.475 0.452 0.423  0.333 0.470 0.426      0.779 0.844   
SVCCA              0.422 0.347 0.306  0.268 0.332 0.796      0.932 1.000   
2nd-Cos            0.733 0.559 0.735  0.365 0.611 0.423      0.950 0.994   
SoftCorr           0.423 0.425 0.424  0.332 0.603 0.449      0.547 0.828   
UnifDiff           0.320 0.334 0.497  0.396 0.531 0.902      0.334 0.531   

Measure                  Violation Rate                                 \
Dataset                            Cora             Flickr               
Model               SAGE            GAT   GCN  SAGE    GAT   GCN  SAGE   
Similarity Measure                                                       
AlignCos           0.699          0.399 0.341 0.336  0.421 0.064 0.500   
CKA                1.000          0.493 0.451 0.488  0.458 0.093 0.139   
ConcDiff           1.000          0.261 0.455 0.219  0.519 0.277 0.152   
DistCorr           1.000          0.479 0.410 0.369  0.482 0.052 0.442   
EOS                0.426          0.342 0.351 0.449  0.419 0.500 0.484   
GULP               0.426          0.341 0.369 0.444  0.429 0.497 0.488   
HardCorr           0.828          0.492 0.473 0.489  0.333 0.059 0.321   
Jaccard            0.432          0.447 0.473 0.492  0.432 0.229 0.421   
LinReg             0.677          0.312 0.287 0.421  0.484 0.511 0.337   
MagDiff            0.490          0.263 0.488 0.396  0.528 0.108 0.066   
ShapeMet           0.726          0.478 0.461 0.487  0.467 0.048 0.396   
OrthProc           0.726          0.478 0.461 0.487  0.467 0.048 0.396   
PermProc           0.724          0.391 0.355 0.286  0.500 0.061 0.031   
ProcDist           0.983          0.403 0.327 0.323  0.490 0.068 0.118   
RSA                0.430          0.461 0.307 0.385  0.363 0.115 0.475   
RankSim            0.552          0.357 0.345 0.465  0.417 0.337 0.470   
SVCCA              1.000          0.409 0.381 0.445  0.457 0.311 0.087   
2nd-Cos            0.960          0.136 0.201 0.131  0.340 0.177 0.499   
SoftCorr           0.818          0.495 0.468 0.489  0.351 0.145 0.329   
UnifDiff           0.539          0.328 0.305 0.185  0.251 0.160 0.037   

Measure                                    
Dataset            OGBN-Arxiv              
Model                     GAT   GCN  SAGE  
Similarity Measure                         
AlignCos                0.027 0.004 0.332  
CKA                     0.000 0.000 0.000  
ConcDiff                0.001 0.008 0.000  
DistCorr                0.000 0.000 0.000  
EOS                     0.494 0.502 0.458  
GU

In [9]:
get_agg_pivot_table("label_test", ["cora", "flickr", "ogbn-arxiv"], groups=2)

Measure            AUPRC                                                  \
Dataset             Cora             Flickr             OGBN-Arxiv         
Model                GAT   GCN  SAGE    GAT   GCN  SAGE        GAT   GCN   
Similarity Measure                                                         
AlignCos           0.636 0.726 0.696  0.518 1.000 0.586      1.000 1.000   
CKA                0.586 0.595 0.586  0.393 1.000 0.969      1.000 1.000   
ConcDiff           0.928 0.340 0.979  0.348 0.708 1.000      1.000 0.946   
DistCorr           0.588 0.605 0.670  0.360 1.000 0.610      1.000 1.000   
EOS                0.741 0.757 0.600  0.333 0.586 0.588      0.586 0.586   
GULP               0.741 0.676 0.603  0.360 0.319 0.587      0.594 0.587   
HardCorr           0.586 0.589 0.586  0.591 1.000 0.830      0.710 1.000   
Jaccard            0.597 0.589 0.586  0.442 0.941 0.600      1.000 1.000   
LinReg             0.752 0.804 0.603  0.364 0.445 0.755      0.594 0.633   
MagDiff            0.861 0.353 0.464  0.296 1.000 1.000      0.555 0.328   
ShapeMet           0.589 0.591 0.586  0.402 1.000 0.675      0.824 1.000   
OrthProc           0.589 0.591 0.586  0.402 1.000 0.675      0.824 1.000   
PermProc           0.586 0.644 0.727  0.304 1.000 1.000      1.000 1.000   
ProcDist           0.591 0.693 0.694  0.291 1.000 1.000      1.000 1.000   
RSA                0.591 0.776 0.604  0.555 1.000 0.588      0.591 1.000   
RankSim            0.783 0.677 0.587  0.543 0.741 0.591      1.000 1.000   
SVCCA              0.592 0.627 0.571  0.400 0.723 0.985      0.942 1.000   
2nd-Cos            1.000 1.000 1.000  0.669 1.000 0.587      1.000 1.000   
SoftCorr           0.586 0.590 0.586  0.592 0.995 0.840      0.621 1.000   
UnifDiff           0.320 0.605 0.971  0.834 0.931 1.000      0.440 0.759   

Measure                  Violation Rate                                 \
Dataset                            Cora             Flickr               
Model               SAGE            GAT   GCN  SAGE    GAT   GCN  SAGE   
Similarity Measure                                                       
AlignCos           0.587          0.282 0.208 0.252  0.406 0.000 0.500   
CKA                1.000          0.500 0.432 0.500  0.478 0.000 0.014   
ConcDiff           1.000          0.034 0.418 0.010  0.510 0.140 0.000   
DistCorr           1.000          0.482 0.380 0.298  0.490 0.000 0.352   
EOS                0.589          0.100 0.138 0.402  0.448 0.500 0.482   
GULP               0.591          0.100 0.192 0.388  0.420 0.496 0.490   
HardCorr           1.000          0.500 0.470 0.500  0.310 0.000 0.142   
Jaccard            0.586          0.416 0.472 0.500  0.408 0.036 0.400   
LinReg             0.939          0.142 0.100 0.396  0.472 0.510 0.228   
MagDiff            0.761          0.056 0.460 0.256  0.526 0.000 0.000   
ShapeMet           1.000          0.468 0.460 0.500  0.482 0.000 0.256   
OrthProc           1.000          0.468 0.460 0.500  0.482 0.000 0.256   
PermProc           1.000          0.500 0.272 0.122  0.530 0.000 0.000   
ProcDist           1.000          0.464 0.210 0.154  0.534 0.000 0.000   
RSA                0.610          0.452 0.168 0.378  0.304 0.000 0.476   
RankSim            0.586          0.164 0.214 0.490  0.368 0.208 0.452   
SVCCA              1.000          0.446 0.262 0.440  0.472 0.196 0.006   
2nd-Cos            1.000          0.000 0.000 0.000  0.290 0.000 0.498   
SoftCorr           1.000          0.500 0.460 0.500  0.326 0.002 0.118   
UnifDiff           0.904          0.508 0.164 0.016  0.092 0.038 0.000   

Measure                                    
Dataset            OGBN-Arxiv              
Model                     GAT   GCN  SAGE  
Similarity Measure                         
AlignCos                0.000 0.000 0.496  
CKA                     0.000 0.000 0.000  
ConcDiff                0.000 0.024 0.000  
DistCorr                0.000 0.000 0.000  
EOS                     0.500 0.500 0.464  
GU

#### Shortcut Test Results

In [10]:
get_agg_pivot_table("shortcut_test", ["cora", "flickr", "ogbn-arxiv"])

Measure            AUPRC                                                  \
Dataset             Cora             Flickr             OGBN-Arxiv         
Model                GAT   GCN  SAGE    GAT   GCN  SAGE        GAT   GCN   
Similarity Measure                                                         
AlignCos           0.117 0.145 0.211  0.251 0.372 0.467      0.895 0.855   
CKA                0.226 0.134 0.303  0.166 0.146 0.659      0.830 0.732   
ConcDiff           0.095 0.151 0.094  0.178 0.135 0.144      0.709 0.610   
DistCorr           0.222 0.142 0.370  0.145 0.160 0.749      0.849 0.897   
EOS                0.085 0.107 0.188  0.317 0.206 0.286      0.602 0.458   
GULP               0.088 0.103 0.188  0.240 0.090 0.286      0.602 0.425   
HardCorr           0.107 0.096 0.180  0.300 0.204 0.435      0.356 0.300   
Jaccard            0.172 0.145 0.347  0.319 0.616 0.504      0.622 0.762   
LinReg             0.157 0.139 0.364  0.191 0.211 0.424      0.800 0.556   
MagDiff            0.091 0.137 0.091  0.113 0.177 0.606      0.704 0.356   
ShapeMet           0.159 0.131 0.345  0.151 0.220 0.590      0.777 0.678   
OrthProc           0.159 0.131 0.345  0.151 0.220 0.590      0.777 0.677   
PermProc           0.100 0.100 0.121  0.139 0.173 0.429      0.397 0.386   
ProcDist           0.149 0.124 0.342  0.150 0.250 0.486      0.598 0.726   
RSA                0.229 0.140 0.161  0.251 0.388 0.698      0.624 0.569   
RSMDiff            0.380 0.196 0.219    NaN   NaN   NaN        NaN   NaN   
RankSim            0.165 0.147 0.341  0.334 0.603 0.454      0.617 0.841   
SVCCA              0.275 0.118 0.170  0.168 0.123 0.560      0.598 0.745   
2nd-Cos            0.201 0.160 0.442  0.313 0.616 0.634      0.830 0.970   
SoftCorr           0.107 0.098 0.195  0.301 0.291 0.416      0.354 0.290   
UnifDiff           0.118 0.095 0.282  0.300 0.341 0.499      0.771 0.723   

Measure                  Violation Rate                                 \
Dataset                            Cora             Flickr               
Model               SAGE            GAT   GCN  SAGE    GAT   GCN  SAGE   
Similarity Measure                                                       
AlignCos           0.635          0.495 0.402 0.249  0.339 0.124 0.110   
CKA                0.525          0.403 0.402 0.198  0.434 0.450 0.040   
ConcDiff           0.727          0.547 0.535 0.573  0.406 0.482 0.385   
DistCorr           0.538          0.410 0.386 0.177  0.382 0.356 0.032   
EOS                0.507          0.593 0.466 0.381  0.304 0.232 0.360   
GULP               0.507          0.585 0.483 0.377  0.291 0.521 0.361   
HardCorr           0.361          0.465 0.507 0.285  0.302 0.252 0.117   
Jaccard            0.599          0.346 0.399 0.217  0.274 0.066 0.126   
LinReg             0.562          0.418 0.438 0.175  0.330 0.340 0.204   
MagDiff            0.525          0.543 0.533 0.572  0.509 0.357 0.139   
ShapeMet           0.729          0.427 0.406 0.181  0.391 0.280 0.075   
OrthProc           0.729          0.427 0.406 0.181  0.391 0.280 0.075   
PermProc           0.182          0.520 0.477 0.411  0.462 0.337 0.144   
ProcDist           0.602          0.437 0.417 0.191  0.448 0.267 0.095   
RSA                0.545          0.328 0.388 0.322  0.272 0.141 0.049   
RSMDiff              NaN          0.192 0.356 0.345    NaN   NaN   NaN   
RankSim            0.619          0.360 0.422 0.222  0.236 0.074 0.138   
SVCCA              0.642          0.384 0.513 0.462  0.427 0.451 0.158   
2nd-Cos            0.852          0.369 0.374 0.137  0.264 0.073 0.046   
SoftCorr           0.358          0.467 0.500 0.268  0.300 0.196 0.137   
UnifDiff           0.526          0.512 0.527 0.234  0.379 0.216 0.201   

Measure                                    
Dataset            OGBN-Arxiv              
Model                     GAT   GCN  SAGE  
Similarity Measure                         
AlignCos                0.009 0.022 0.119  
CKA                     0.02

In [11]:
shortcut_test_df = get_agg_pivot_table("shortcut_test", ["cora", "flickr", "ogbn-arxiv"], groups=3)
shortcut_test_df.to_latex("graphs_results_shortcut_test.tex", 
                       **LATEX_FORMAT_DICT
                      )
shortcut_test_df

Measure            AUPRC                                                  \
Dataset             Cora             Flickr             OGBN-Arxiv         
Model                GAT   GCN  SAGE    GAT   GCN  SAGE        GAT   GCN   
Similarity Measure                                                         
AlignCos           0.303 0.319 0.599  0.480 0.895 1.000      1.000 1.000   
CKA                0.463 0.273 0.777  0.330 0.282 1.000      1.000 1.000   
ConcDiff           0.204 0.182 0.169  0.324 0.182 0.176      1.000 0.815   
DistCorr           0.459 0.284 0.820  0.317 0.333 1.000      1.000 1.000   
EOS                0.163 0.217 0.495  0.544 0.461 0.433      0.968 1.000   
GULP               0.172 0.197 0.496  0.451 0.196 0.433      0.965 0.986   
HardCorr           0.219 0.200 0.347  0.519 0.549 1.000      0.835 0.798   
Jaccard            0.380 0.305 0.782  0.545 1.000 0.828      1.000 1.000   
LinReg             0.334 0.299 0.742  0.357 0.365 0.609      1.000 1.000   
MagDiff            0.187 0.267 0.151  0.198 0.266 0.783      1.000 0.550   
ShapeMet           0.366 0.274 0.794  0.307 0.581 1.000      1.000 1.000   
OrthProc           0.366 0.274 0.794  0.307 0.581 1.000      1.000 1.000   
PermProc           0.225 0.207 0.268  0.272 0.418 1.000      0.766 0.649   
ProcDist           0.370 0.247 0.812  0.331 0.599 1.000      1.000 1.000   
RSA                0.578 0.299 0.432  0.521 0.894 1.000      0.902 0.966   
RankSim            0.362 0.332 0.645  0.538 1.000 0.774      1.000 1.000   
SVCCA              0.571 0.191 0.357  0.315 0.239 0.929      0.825 1.000   
2nd-Cos            0.472 0.343 0.907  0.522 1.000 1.000      1.000 1.000   
SoftCorr           0.218 0.209 0.404  0.571 0.663 1.000      0.830 0.816   
UnifDiff           0.188 0.192 0.637  0.480 0.593 0.499      0.974 0.991   

Measure                  Violation Rate                                 \
Dataset                            Cora             Flickr               
Model               SAGE            GAT   GCN  SAGE    GAT   GCN  SAGE   
Similarity Measure                                                       
AlignCos           1.000          0.411 0.305 0.119  0.231 0.019 0.000   
CKA                0.981          0.364 0.339 0.097  0.348 0.428 0.000   
ConcDiff           0.962          0.502 0.532 0.573  0.393 0.503 0.537   
DistCorr           0.994          0.378 0.312 0.084  0.282 0.313 0.000   
EOS                0.724          0.575 0.426 0.260  0.191 0.128 0.400   
GULP               0.724          0.561 0.475 0.261  0.189 0.508 0.400   
HardCorr           0.724          0.419 0.441 0.232  0.192 0.171 0.000   
Jaccard            1.000          0.286 0.303 0.071  0.160 0.000 0.037   
LinReg             1.000          0.376 0.383 0.053  0.277 0.297 0.189   
MagDiff            0.535          0.511 0.519 0.585  0.523 0.331 0.114   
ShapeMet           1.000          0.380 0.325 0.085  0.279 0.202 0.000   
OrthProc           1.000          0.380 0.325 0.085  0.279 0.202 0.000   
PermProc           0.433          0.481 0.412 0.328  0.424 0.281 0.000   
ProcDist           0.997          0.396 0.355 0.087  0.392 0.185 0.000   
RSA                0.978          0.225 0.298 0.216  0.187 0.030 0.000   
RankSim            1.000          0.277 0.345 0.105  0.170 0.000 0.078   
SVCCA              0.970          0.301 0.538 0.403  0.337 0.433 0.027   
2nd-Cos            1.000          0.311 0.256 0.029  0.179 0.000 0.000   
SoftCorr           0.724          0.421 0.430 0.211  0.176 0.133 0.000   
UnifDiff           0.717          0.525 0.482 0.089  0.327 0.169 0.234   

Measure                                    
Dataset            OGBN-Arxiv              
Model                     GAT   GCN  SAGE  
Similarity Measure                         
AlignCos                0.000 0.000 0.000  
CKA                     0.000 0.000 0.004  
ConcDiff                0.000 0.037 0.009  
DistCorr                0.000 0.000 0.001  
EOS                     0.007 0.000 0.167  
GU

In [12]:
get_agg_pivot_table("shortcut_test", ["cora", "flickr", "ogbn-arxiv"], groups=2)

Measure            AUPRC                                                  \
Dataset             Cora             Flickr             OGBN-Arxiv         
Model                GAT   GCN  SAGE    GAT   GCN  SAGE        GAT   GCN   
Similarity Measure                                                         
AlignCos           0.665 0.875 0.990  0.743 1.000 1.000      1.000 1.000   
CKA                0.705 0.723 1.000  0.658 0.509 1.000      1.000 1.000   
ConcDiff           0.314 0.281 0.312  0.490 0.350 0.359      1.000 1.000   
DistCorr           0.698 0.809 1.000  0.599 0.684 1.000      1.000 1.000   
EOS                0.312 0.425 0.946  0.918 1.000 0.586      1.000 1.000   
GULP               0.324 0.351 0.937  0.965 0.351 0.586      1.000 1.000   
HardCorr           0.435 0.424 1.000  0.992 1.000 1.000      1.000 1.000   
Jaccard            0.814 0.834 1.000  1.000 1.000 1.000      1.000 1.000   
LinReg             0.663 0.663 1.000  0.569 0.530 0.827      1.000 1.000   
MagDiff            0.298 0.484 0.264  0.326 0.655 1.000      1.000 1.000   
ShapeMet           0.700 0.741 1.000  0.651 0.921 1.000      1.000 1.000   
OrthProc           0.700 0.741 1.000  0.651 0.921 1.000      1.000 1.000   
PermProc           0.453 0.516 0.765  0.380 0.871 1.000      1.000 1.000   
ProcDist           0.683 0.666 1.000  0.444 0.939 1.000      1.000 1.000   
RSA                0.990 0.751 0.960  0.965 1.000 1.000      1.000 1.000   
RankSim            0.804 0.850 1.000  1.000 1.000 1.000      1.000 1.000   
SVCCA              0.848 0.306 0.553  0.603 0.453 0.915      0.932 1.000   
2nd-Cos            0.783 0.912 1.000  0.995 1.000 1.000      1.000 1.000   
SoftCorr           0.438 0.435 1.000  0.995 1.000 1.000      1.000 1.000   
UnifDiff           0.330 0.365 1.000  0.676 1.000 0.965      1.000 1.000   

Measure                  Violation Rate                                 \
Dataset                            Cora             Flickr               
Model               SAGE            GAT   GCN  SAGE    GAT   GCN  SAGE   
Similarity Measure                                                       
AlignCos           1.000          0.222 0.060 0.004  0.144 0.000 0.000   
CKA                1.000          0.248 0.164 0.000  0.268 0.366 0.000   
ConcDiff           1.000          0.488 0.572 0.560  0.424 0.470 0.522   
DistCorr           1.000          0.258 0.130 0.000  0.184 0.214 0.000   
EOS                1.000          0.534 0.298 0.020  0.030 0.000 0.500   
GULP               1.000          0.522 0.384 0.024  0.014 0.472 0.500   
HardCorr           1.000          0.262 0.306 0.000  0.004 0.000 0.000   
Jaccard            1.000          0.056 0.076 0.000  0.000 0.000 0.000   
LinReg             1.000          0.226 0.200 0.000  0.176 0.298 0.066   
MagDiff            1.000          0.510 0.530 0.580  0.538 0.132 0.000   
ShapeMet           1.000          0.230 0.142 0.000  0.176 0.060 0.000   
OrthProc           1.000          0.230 0.142 0.000  0.176 0.060 0.000   
PermProc           0.608          0.416 0.226 0.154  0.436 0.058 0.000   
ProcDist           1.000          0.268 0.182 0.000  0.404 0.038 0.000   
RSA                1.000          0.004 0.102 0.016  0.020 0.000 0.000   
RankSim            1.000          0.062 0.100 0.000  0.000 0.000 0.000   
SVCCA              1.000          0.178 0.536 0.374  0.240 0.372 0.052   
2nd-Cos            1.000          0.124 0.058 0.000  0.002 0.000 0.000   
SoftCorr           1.000          0.256 0.304 0.000  0.002 0.000 0.000   
UnifDiff           0.891          0.494 0.416 0.000  0.260 0.000 0.020   

Measure                                    
Dataset            OGBN-Arxiv              
Model                     GAT   GCN  SAGE  
Similarity Measure                         
AlignCos                0.000 0.000 0.000  
CKA                     0.000 0.000 0.000  
ConcDiff                0.000 0.000 0.000  
DistCorr                0.000 0.000 0.000  
EOS                     0.000 0.000 0.000  
GU

#### Augmentation Test Results

In [13]:
get_agg_pivot_table("augmentation_test", ["cora", "flickr", "ogbn-arxiv"])

Measure            AUPRC                                                  \
Dataset             Cora             Flickr             OGBN-Arxiv         
Model                GAT   GCN  SAGE    GAT   GCN  SAGE        GAT   GCN   
Similarity Measure                                                         
AlignCos           0.319 0.207 0.313  0.446 0.778 0.268      0.667 0.314   
CKA                0.461 0.317 0.334  0.265 0.710 0.345      0.624 0.602   
ConcDiff           0.296 0.105 0.302  0.256 0.208 0.178      0.287 0.177   
DistCorr           0.460 0.380 0.336  0.336 0.712 0.371      0.643 0.591   
EOS                0.136 0.282 0.324  0.302 0.397 0.231      0.465 0.370   
GULP               0.138 0.208 0.326  0.462 0.120 0.238      0.502 0.148   
HardCorr           0.277 0.243 0.322  0.444 0.690 0.305      0.184 0.139   
Jaccard            0.473 0.377 0.354  0.539 0.846 0.459      0.758 0.566   
LinReg             0.407 0.457 0.379  0.284 0.186 0.345      0.495 0.229   
MagDiff            0.347 0.128 0.173  0.270 0.460 0.094      0.393 0.404   
ShapeMet           0.425 0.352 0.347  0.285 0.739 0.341      0.500 0.249   
OrthProc           0.425 0.352 0.347  0.285 0.739 0.341      0.500 0.249   
PermProc           0.265 0.349 0.300  0.266 0.678 0.263      0.449 0.391   
ProcDist           0.367 0.447 0.331  0.288 0.789 0.361      0.954 0.503   
RSA                0.342 0.187 0.245  0.562 0.802 0.358      0.421 0.416   
RankSim            0.448 0.312 0.328  0.493 0.847 0.462      0.758 0.574   
SVCCA              0.156 0.162 0.160  0.261 0.753 0.250      0.398 0.506   
2nd-Cos            0.497 0.366 0.368  0.641 0.866 0.411      0.785 0.580   
SoftCorr           0.272 0.288 0.333  0.336 0.671 0.297      0.172 0.135   
UnifDiff           0.843 0.316 0.318  0.161 0.744 0.162      0.316 0.456   

Measure                  Violation Rate                                 \
Dataset                            Cora             Flickr               
Model               SAGE            GAT   GCN  SAGE    GAT   GCN  SAGE   
Similarity Measure                                                       
AlignCos           0.171          0.163 0.335 0.276  0.193 0.049 0.187   
CKA                0.642          0.104 0.234 0.214  0.237 0.072 0.149   
ConcDiff           0.248          0.272 0.493 0.356  0.269 0.342 0.387   
DistCorr           0.631          0.096 0.203 0.214  0.193 0.064 0.169   
EOS                0.175          0.341 0.276 0.233  0.302 0.165 0.341   
GULP               0.176          0.336 0.332 0.225  0.161 0.486 0.336   
HardCorr           0.148          0.320 0.279 0.240  0.168 0.072 0.222   
Jaccard            0.439          0.081 0.183 0.182  0.134 0.035 0.153   
LinReg             0.205          0.103 0.096 0.149  0.261 0.466 0.220   
MagDiff            0.995          0.262 0.451 0.391  0.342 0.165 0.540   
ShapeMet           0.244          0.117 0.199 0.194  0.224 0.061 0.215   
OrthProc           0.244          0.117 0.199 0.194  0.224 0.061 0.215   
PermProc           0.324          0.530 0.313 0.298  0.352 0.071 0.172   
ProcDist           0.591          0.166 0.232 0.227  0.323 0.043 0.101   
RSA                0.199          0.190 0.404 0.321  0.160 0.048 0.218   
RankSim            0.483          0.102 0.324 0.234  0.142 0.033 0.147   
SVCCA              0.504          0.480 0.506 0.482  0.267 0.065 0.232   
2nd-Cos            0.501          0.072 0.198 0.181  0.137 0.028 0.120   
SoftCorr           0.161          0.311 0.257 0.220  0.169 0.074 0.286   
UnifDiff           0.347          0.016 0.238 0.235  0.340 0.046 0.498   

Measure                                    
Dataset            OGBN-Arxiv              
Model                     GAT   GCN  SAGE  
Similarity Measure                         
AlignCos                0.069 0.152 0.511  
CKA                     0.080 0.074 0.049  
ConcDiff                0.251 0.299 0.240  
DistCorr                0.074 0.079 0.058  
EOS                     0.093 0.165 0.419  
GU

In [14]:
aug_test_df = get_agg_pivot_table("augmentation_test", ["cora", "flickr", "ogbn-arxiv"], groups=3)
aug_test_df.to_latex("graphs_results_augmentation_test.tex", 
                       **LATEX_FORMAT_DICT
                      )
aug_test_df

Measure            AUPRC                                                  \
Dataset             Cora             Flickr             OGBN-Arxiv         
Model                GAT   GCN  SAGE    GAT   GCN  SAGE        GAT   GCN   
Similarity Measure                                                         
AlignCos           0.908 0.510 0.635  0.544 0.686 0.698      1.000 0.736   
CKA                0.906 0.697 0.731  0.508 0.572 0.747      0.987 0.997   
ConcDiff           0.623 0.174 0.504  0.534 0.412 0.348      0.508 0.426   
DistCorr           0.891 0.785 0.732  0.609 0.602 0.794      0.989 1.000   
EOS                0.342 0.535 0.618  0.459 0.679 0.529      1.000 0.819   
GULP               0.349 0.448 0.612  0.555 0.223 0.542      1.000 0.557   
HardCorr           0.513 0.512 0.633  0.571 0.712 0.716      0.536 0.466   
Jaccard            0.953 0.781 0.807  0.580 0.743 0.880      1.000 1.000   
LinReg             0.872 0.943 0.850  0.338 0.302 0.805      0.809 0.724   
MagDiff            0.535 0.204 0.390  0.430 0.588 0.174      0.693 0.771   
ShapeMet           0.835 0.812 0.784  0.540 0.627 0.760      0.744 0.724   
OrthProc           0.835 0.812 0.784  0.540 0.627 0.760      0.744 0.724   
PermProc           0.423 0.701 0.613  0.398 0.649 0.685      0.936 0.881   
ProcDist           0.749 0.764 0.693  0.401 0.696 0.814      1.000 1.000   
RSA                0.784 0.385 0.573  0.586 0.680 0.718      0.674 0.745   
RankSim            0.947 0.540 0.618  0.553 0.746 0.882      1.000 1.000   
SVCCA              0.339 0.267 0.294  0.495 0.643 0.673      0.820 0.889   
2nd-Cos            0.967 0.734 0.863  0.622 0.777 0.920      1.000 1.000   
SoftCorr           0.496 0.630 0.702  0.546 0.729 0.581      0.448 0.425   
UnifDiff           1.000 0.747 0.534  0.247 0.683 0.239      0.447 0.757   

Measure                  Violation Rate                                 \
Dataset                            Cora             Flickr               
Model               SAGE            GAT   GCN  SAGE    GAT   GCN  SAGE   
Similarity Measure                                                       
AlignCos           0.427          0.022 0.217 0.119  0.192 0.130 0.099   
CKA                1.000          0.035 0.103 0.081  0.264 0.173 0.088   
ConcDiff           0.533          0.133 0.508 0.219  0.258 0.221 0.319   
DistCorr           1.000          0.035 0.071 0.088  0.188 0.162 0.059   
EOS                0.488          0.248 0.143 0.104  0.297 0.125 0.203   
GULP               0.484          0.237 0.210 0.109  0.161 0.477 0.191   
HardCorr           0.506          0.195 0.151 0.105  0.149 0.106 0.084   
Jaccard            0.991          0.016 0.096 0.054  0.159 0.115 0.027   
LinReg             0.724          0.048 0.017 0.037  0.309 0.473 0.067   
MagDiff            1.000          0.173 0.485 0.258  0.341 0.132 0.551   
ShapeMet           0.724          0.048 0.050 0.069  0.232 0.152 0.063   
OrthProc           0.724          0.048 0.050 0.069  0.232 0.152 0.063   
PermProc           0.734          0.486 0.220 0.113  0.384 0.128 0.099   
ProcDist           1.000          0.103 0.115 0.095  0.374 0.113 0.051   
RSA                0.495          0.062 0.319 0.156  0.159 0.143 0.080   
RankSim            1.000          0.017 0.278 0.120  0.165 0.111 0.039   
SVCCA              0.813          0.421 0.457 0.449  0.316 0.162 0.136   
2nd-Cos            0.992          0.011 0.125 0.028  0.164 0.092 0.019   
SoftCorr           0.460          0.204 0.114 0.077  0.163 0.128 0.148   
UnifDiff           0.526          0.000 0.089 0.141  0.341 0.097 0.482   

Measure                                    
Dataset            OGBN-Arxiv              
Model                     GAT   GCN  SAGE  
Similarity Measure                         
AlignCos                0.000 0.129 0.464  
CKA                     0.003 0.001 0.000  
ConcDiff                0.160 0.240 0.196  
DistCorr                0.003 0.000 0.000  
EOS                     0.000 0.035 0.250  
GU

In [15]:
get_agg_pivot_table("augmentation_test", ["cora", "flickr", "ogbn-arxiv"], groups=2)

Measure            AUPRC                                                  \
Dataset             Cora             Flickr             OGBN-Arxiv         
Model                GAT   GCN  SAGE    GAT   GCN  SAGE        GAT   GCN   
Similarity Measure                                                         
AlignCos           1.000 0.979 0.990  0.899 1.000 1.000      1.000 1.000   
CKA                1.000 1.000 1.000  0.827 0.995 1.000      1.000 1.000   
ConcDiff           0.906 0.306 0.880  0.808 0.882 0.517      0.784 0.899   
DistCorr           1.000 1.000 1.000  0.924 1.000 1.000      1.000 1.000   
EOS                0.995 0.995 1.000  0.671 0.946 1.000      1.000 1.000   
GULP               0.995 0.808 1.000  1.000 0.360 1.000      1.000 0.797   
HardCorr           0.986 1.000 1.000  0.990 1.000 1.000      0.995 0.586   
Jaccard            1.000 1.000 1.000  1.000 1.000 1.000      1.000 1.000   
LinReg             1.000 1.000 1.000  0.754 0.347 1.000      1.000 1.000   
MagDiff            1.000 0.360 0.447  0.627 1.000 0.320      1.000 1.000   
ShapeMet           1.000 1.000 1.000  0.832 1.000 1.000      1.000 1.000   
OrthProc           1.000 1.000 1.000  0.832 1.000 1.000      1.000 1.000   
PermProc           0.587 1.000 0.983  0.560 1.000 1.000      1.000 1.000   
ProcDist           1.000 1.000 1.000  0.565 1.000 1.000      1.000 1.000   
RSA                1.000 0.845 0.971  0.940 1.000 1.000      0.988 1.000   
RankSim            1.000 0.776 0.937  1.000 1.000 1.000      1.000 1.000   
SVCCA              0.580 0.477 0.509  0.798 0.986 0.961      1.000 0.857   
2nd-Cos            1.000 1.000 1.000  1.000 1.000 1.000      1.000 1.000   
SoftCorr           1.000 1.000 1.000  0.983 1.000 1.000      0.792 0.586   
UnifDiff           1.000 0.881 0.983  0.488 1.000 0.324      0.914 0.698   

Measure                  Violation Rate                                 \
Dataset                            Cora             Flickr               
Model               SAGE            GAT   GCN  SAGE    GAT   GCN  SAGE   
Similarity Measure                                                       
AlignCos           0.593          0.000 0.010 0.004  0.048 0.000 0.000   
CKA                1.000          0.000 0.000 0.000  0.108 0.002 0.000   
ConcDiff           0.988          0.050 0.474 0.088  0.208 0.066 0.248   
DistCorr           1.000          0.000 0.000 0.000  0.042 0.000 0.000   
EOS                1.000          0.002 0.002 0.000  0.186 0.030 0.000   
GULP               1.000          0.002 0.080 0.000  0.000 0.440 0.000   
HardCorr           1.000          0.008 0.000 0.000  0.004 0.000 0.000   
Jaccard            1.000          0.000 0.000 0.000  0.000 0.000 0.000   
LinReg             1.000          0.000 0.000 0.000  0.132 0.476 0.000   
MagDiff            1.000          0.000 0.516 0.298  0.308 0.000 0.546   
ShapeMet           1.000          0.000 0.000 0.000  0.098 0.000 0.000   
OrthProc           1.000          0.000 0.000 0.000  0.098 0.000 0.000   
PermProc           1.000          0.498 0.000 0.010  0.336 0.000 0.000   
ProcDist           1.000          0.000 0.000 0.000  0.318 0.000 0.000   
RSA                0.931          0.000 0.098 0.014  0.036 0.000 0.000   
RankSim            1.000          0.000 0.188 0.018  0.000 0.000 0.000   
SVCCA              0.793          0.284 0.400 0.354  0.208 0.008 0.028   
2nd-Cos            1.000          0.000 0.000 0.000  0.000 0.000 0.000   
SoftCorr           1.000          0.000 0.000 0.000  0.008 0.000 0.000   
UnifDiff           0.828          0.000 0.108 0.008  0.236 0.000 0.502   

Measure                                    
Dataset            OGBN-Arxiv              
Model                     GAT   GCN  SAGE  
Similarity Measure                         
AlignCos                0.000 0.000 0.438  
CKA                     0.000 0.000 0.000  
ConcDiff                0.108 0.050 0.006  
DistCorr                0.000 0.000 0.000  
EOS                     0.000 0.000 0.000  
GU

#### Layer Monotonicity Test Results

In [16]:
layer_test_df = get_agg_pivot_table("layer_test", ["cora", "flickr", "ogbn-arxiv"])
layer_test_df.to_latex("graphs_results_layer_test.tex", 
                       **LATEX_FORMAT_DICT
                      )
layer_test_df

Measure            Spearman Correlation                                  \
Dataset                            Cora             Flickr                
Model                               GAT   GCN  SAGE    GAT   GCN   SAGE   
Similarity Measure                                                        
AlignCos                          1.000 0.980 1.000  0.040 0.327  0.680   
CKA                               1.000 0.980 1.000  0.387 0.603  0.890   
ConcDiff                          0.403 0.850 0.250  0.243 0.383 -0.273   
DistCorr                          1.000 1.000 1.000  0.467 0.630  0.993   
EOS                               1.000 1.000 1.000  0.917 1.000  1.000   
GULP                              1.000 1.000 1.000  0.330 0.540  1.000   
HardCorr                          0.540 0.833 0.983  0.213 0.627  0.803   
Jaccard                           1.000 1.000 1.000  0.953 0.950  0.973   
LinReg                            1.000 1.000 1.000  0.030 0.450  1.000   
MagDiff                           0.893 0.550 0.490  0.383 0.583  0.500   
ShapeMet                          1.000 1.000 1.000  0.453 0.620  0.983   
OrthProc                          1.000 1.000 1.000  0.453 0.620  0.983   
PermProc                          1.000 0.917 1.000  0.630 0.603  0.680   
ProcDist                          1.000 0.993 1.000  0.617 0.660  1.000   
RSA                               1.000 0.837 0.910  0.437 0.580  0.987   
RankSim                           1.000 1.000 1.000  0.987 0.950  0.967   
SVCCA                             0.613 0.410 0.777  0.467 0.533  0.440   
2nd-Cos                           1.000 1.000 1.000  0.517 0.810  0.957   
SoftCorr                          0.500 0.957 0.993  0.113 0.787  0.893   
UnifDiff                          0.590 0.667 0.667  0.820 0.820  0.890   

Measure                                     Violation Rate                     \
Dataset            OGBN-Arxiv                         Cora             Flickr   
Model                     GAT    GCN   SAGE            GAT   GCN  SAGE    GAT   
Similarity Measure                                                              
AlignCos                0.837  0.843  0.933          0.000 0.030 0.000  0.540   
CKA                     0.960  0.850  0.457          0.000 0.020 0.000  0.410   
ConcDiff                0.397  0.733  0.390          0.380 0.150 0.480  0.420   
DistCorr                0.933  0.807  0.523          0.000 0.000 0.000  0.370   
EOS                     1.000  1.000  1.000          0.000 0.000 0.000  0.080   
GULP                    1.000  0.733  1.000          0.000 0.000 0.000  0.280   
HardCorr                1.000  0.833  0.620          0.240 0.090 0.010  0.320   
Jaccard                 1.000  0.973  0.987          0.000 0.000 0.000  0.040   
LinReg                  1.000  0.993  1.000          0.000 0.000 0.000  0.410   
MagDiff                 0.633  0.060  0.290          0.080 0.370 0.280  0.420   
ShapeMet                0.993  0.967  0.993          0.000 0.000 0.000  0.340   
OrthProc                0.993  0.967  0.993          0.000 0.000 0.000  0.340   
PermProc                1.000  0.243  0.883          0.000 0.090 0.000  0.250   
ProcDist                1.000  0.833  0.853          0.000 0.010 0.000  0.220   
RSA                     0.940  0.523  0.973          0.000 0.100 0.030  0.300   
RankSim                 1.000  0.993  0.987          0.000 0.000 0.000  0.020   
SVCCA                   0.327 -0.173 -0.097          0.190 0.370 0.200  0.430   
2nd-Cos                 1.000  0.967  1.000          0.000 0.000 0.000  0.330   
SoftCorr                1.000  0.913  0.623          0.220 0.050 0.010  0.360   
UnifDiff                0.677  0.327 -0.447          0.310 0.300 0.300  0.060   

Measure                                                
Dataset                        OGBN-Arxiv              
Model                GCN  SAGE        GAT   GCN  SAGE  
Similarity Measure                                     
AlignCos           0.410 0.230

### Results of Output Correlations

In [17]:
def OUTPUT_DF_FILE_NAME(dataset):
    return f"output_correlation_test_{dataset}_full.csv"

In [18]:
def get_output_correlation_table(datasets, corr_func="spearmanr", acc_test = False):
    dfs = []
    for dataset in datasets:
        path = os.path.join(EXPERIMENT_RESULTS_PATH, OUTPUT_DF_FILE_NAME(dataset))
        df = pd.read_csv(path)
        df = df.loc[df.loc[:,"quality_measure"] == corr_func]
        data = df.loc[:, ["similarity_measure", "functional_similarity_measure", "corr", "architecture"]]
        data = data.rename(COLUMN_NAME_DICT, axis="columns")
        data["Dataset"] = dataset
        dfs.append(data.iloc[:])

    df_cc = pd.concat(dfs, axis=0)
    df_res = df_cc.pivot_table(index="Similarity Measure", columns=["Measure", "Dataset", "Model"], values="corr", aggfunc="mean")
    df_res = df_res.rename(MEASURE_NAME_DICT, axis="index")
    df_res = df_res.rename(PIVOT_COL_DICT, axis="columns")
    if acc_test: 
        return df_res.loc[:,"AbsoluteAccDiff"]
        
    return df_res.loc[:,["Disagreement", "JSD"]]

In [19]:
output_corr_df = get_output_correlation_table(["cora", "flickr", "ogbn-arxiv"])
output_corr_df.to_latex("graphs_results_output_correlation_test.tex", 
                       **LATEX_FORMAT_DICT
                      )
output_corr_df

Measure            Disagreement                                                \
Dataset                    Cora               Flickr               OGBN-Arxiv   
Model                       GAT    GCN   SAGE    GAT    GCN   SAGE        GAT   
Similarity Measure                                                              
AlignCos                 -0.051 -0.084  0.271 -0.083  0.147  0.374      0.166   
CKA                       0.445  0.363  0.001  0.061 -0.211  0.534      0.227   
ConcDiff                  0.030 -0.038 -0.097  0.029 -0.214 -0.043     -0.160   
DistCorr                  0.456  0.529 -0.078 -0.031  0.172  0.395      0.204   
EOS                      -0.088  0.127  0.023  0.231  0.025  0.334      0.119   
GULP                     -0.129 -0.165 -0.090 -0.006  0.029  0.334      0.099   
HardCorr                  0.261  0.454  0.104 -0.093  0.395  0.458      0.239   
Jaccard                   0.120  0.436  0.333 -0.004  0.037  0.419      0.219   
LinReg                   -0.062  0.574  0.090  0.063  0.119  0.457      0.221   
MagDiff                  -0.069 -0.073 -0.253 -0.198  0.056  0.065      0.216   
ShapeMet                  0.053  0.616  0.156  0.034  0.193  0.573      0.268   
OrthProc                  0.053  0.616  0.156  0.034  0.192  0.573      0.268   
PermProc                 -0.138  0.235  0.328 -0.272  0.178 -0.097      0.221   
ProcDist                 -0.175  0.450  0.237 -0.265 -0.134 -0.148      0.303   
RSA                      -0.018  0.055  0.328  0.063  0.348  0.586      0.454   
RankSim                   0.442  0.343  0.227  0.047  0.055  0.237      0.096   
SVCCA                     0.166 -0.057  0.045  0.028 -0.269  0.214     -0.146   
2nd-Cos                   0.299  0.399  0.252 -0.103  0.301  0.143      0.064   
SoftCorr                  0.069  0.485  0.057  0.017  0.390  0.486      0.353   
UnifDiff                 -0.146  0.067  0.161  0.182 -0.340  0.038      0.060   

Measure                             JSD                                     \
Dataset                            Cora               Flickr                 
Model                 GCN   SAGE    GAT    GCN   SAGE    GAT    GCN   SAGE   
Similarity Measure                                                           
AlignCos           -0.098  0.001  0.174  0.017  0.384 -0.013  0.306  0.443   
CKA                 0.029 -0.041  0.535  0.435 -0.031  0.166  0.029  0.576   
ConcDiff           -0.254  0.072  0.014  0.032  0.041  0.028 -0.172 -0.034   
DistCorr            0.110  0.081  0.601  0.597  0.053  0.031  0.456  0.431   
EOS                -0.255 -0.023 -0.036  0.168  0.224  0.114 -0.027  0.385   
GULP                0.154 -0.037 -0.042 -0.096  0.182  0.128  0.149  0.383   
HardCorr            0.020 -0.241  0.516  0.479  0.162  0.095  0.534  0.505   
Jaccard             0.142  0.012  0.379  0.441  0.453  0.107  0.331  0.421   
LinReg             -0.099 -0.188  0.190  0.580  0.331  0.176  0.152  0.477   
MagDiff            -0.252 -0.195 -0.145  0.004  0.063 -0.262  0.032  0.056   
ShapeMet            0.018 -0.165  0.282  0.695  0.272  0.130  0.432  0.626   
OrthProc            0.018 -0.165  0.282  0.695  0.272  0.130  0.432  0.626   
PermProc            0.250 -0.412  0.133  0.289  0.096 -0.424  0.289 -0.099   
ProcDist            0.257 -0.382  0.046  0.516  0.085 -0.379  0.023 -0.180   
RSA                 0.170  0.155  0.200  0.117  0.133  0.144  0.522  0.630   
RankSim             0.027  0.274  0.505  0.363  0.561  0.213  0.328  0.297   
SVCCA               0.085  0.113  0.261 -0.085  0.018  0.130 -0.036  0.232   
2nd-Cos             0.042  0.265  0.529  0.429  0.536  0.068  0.467  0.154   
SoftCorr            0.034 -0.260  0.215  0.534 -0.013  0.226  0.533  0.525   
UnifDiff           -0.330  0.097 -0.114  0.133  0.270  0.212 -0.318  0.016   

Measure                                      
Dataset            OGBN-Arxiv                
Model                     GAT    GCN   SAGE  
Similarity Measure                          

In [20]:
acc_corr_df = get_output_correlation_table(["cora", "flickr", "ogbn-arxiv"], acc_test=True)
acc_corr_df.to_latex("graphs_results_accuracy_correlation_test.tex", 
                       **LATEX_FORMAT_DICT
                      )
acc_corr_df

Dataset              Cora               Flickr               OGBN-Arxiv  \
Model                 GAT    GCN   SAGE    GAT    GCN   SAGE        GAT   
Similarity Measure                                                        
AlignCos           -0.369 -0.188  0.154  0.088  0.173  0.388     -0.185   
CKA                -0.080 -0.310 -0.134 -0.095 -0.150  0.372     -0.129   
ConcDiff           -0.098 -0.021 -0.213  0.078 -0.110 -0.060     -0.191   
DistCorr           -0.112 -0.299 -0.126 -0.041  0.200  0.360     -0.127   
EOS                -0.002  0.290  0.085  0.399  0.227  0.224      0.048   
GULP               -0.037  0.245  0.085 -0.087  0.041  0.215      0.052   
HardCorr           -0.261  0.030 -0.116 -0.145  0.177  0.323      0.064   
Jaccard            -0.226 -0.042 -0.100 -0.057  0.079  0.074     -0.145   
LinReg             -0.205  0.009 -0.206 -0.115 -0.228  0.277      0.049   
MagDiff            -0.073  0.051 -0.106  0.101  0.178 -0.006     -0.101   
ShapeMet           -0.237 -0.273 -0.263 -0.093  0.187  0.427     -0.175   
OrthProc           -0.237 -0.273 -0.263 -0.093  0.188  0.427     -0.175   
PermProc           -0.317  0.058  0.166  0.094  0.349 -0.041     -0.095   
ProcDist           -0.259 -0.047 -0.008  0.090  0.007  0.013     -0.219   
RSA                -0.331 -0.143  0.064  0.025  0.332  0.300     -0.241   
RankSim             0.035 -0.041 -0.093 -0.049  0.082  0.144     -0.312   
SVCCA              -0.122 -0.039 -0.098 -0.101 -0.210 -0.119      0.119   
2nd-Cos            -0.207 -0.130  0.053 -0.000  0.393 -0.089     -0.210   
SoftCorr           -0.092 -0.130 -0.061 -0.200  0.116  0.308      0.025   
UnifDiff            0.006 -0.125 -0.033  0.067 -0.177  0.007     -0.158   

Dataset                           
Model                 GCN   SAGE  
Similarity Measure                
AlignCos           -0.069  0.201  
CKA                 0.040  0.056  
ConcDiff           -0.247 -0.004  
DistCorr            0.138  0.010  
EOS                -0.050  0.010  
GULP               -0.185 -0.013  
HardCorr            0.132 -0.040  
Jaccard            -0.275 -0.132  
LinReg             -0.229 -0.021  
MagDiff            -0.214  0.033  
ShapeMet           -0.126 -0.065  
OrthProc           -0.126 -0.065  
PermProc           -0.052  0.379  
ProcDist           -0.238  0.349  
RSA                 0.560  0.171  
RankSim            -0.257 -0.004  
SVCCA              -0.179 -0.034  
2nd-Cos            -0.415 -0.093  
SoftCorr            0.067 -0.016  
UnifDiff           -0.175 -0.041

### CD Plots

In [21]:
# autorank package needed for creation of CD plots 
from autorank import autorank, plot_stats, create_report, latex_table
from autorank._util import *
from matplotlib import pyplot as plt

In [22]:
def get_figsize(columnwidth, wf=0.5, hf=(5. ** 0.5 - 1.0) / 2.0):
    """ Credit: https://stackoverflow.com/a/31527287
    Parameters:
      - wf [float]:  width fraction in columnwidth units
      - hf [float]:  height fraction in columnwidth units.
                     Set by default to golden ratio.
      - columnwidth [float]: width of the column in latex. Get this from LaTeX
                             using \showthe\columnwidth
    Returns:  [fig_width,fig_height]: that should be given to matplotlib
    """
    fig_width_pt = columnwidth * wf
    inches_per_pt = 1.0 / 72.27  # Convert pt to inch
    fig_width = fig_width_pt * inches_per_pt  # width in inches
    fig_height = fig_width * hf  # height in inches
    return fig_width, fig_height

PLOTS_BASE_WIDTH = 433.62  # pt
PLOTS_CD_WIDTH, PLOTS_CD_HEIGHT = get_figsize(PLOTS_BASE_WIDTH, wf=1.5)

In [23]:
def get_autorank_df(experiments=EXPERIMENTS, datasets=DATASETS, quality_measure = "violation_rate"):
    dfs = []
    for experiment in experiments:
        df = get_agg_pivot_table(experiment, datasets)        
        if experiment == "layer_test":
            dfs.append(df.loc[:,"Spearman Correlation"].dropna())
        else:
            dfs.append(df.loc[:,"AUPRC"].dropna())
    df_out = get_output_correlation_table(datasets).dropna()
    dfs.append(df_out.loc[:,"JSD"])
    df_acc = get_output_correlation_table(datasets, acc_test=True).dropna()
    dfs.append(df_acc)
    return pd.concat(dfs, axis=1)


In [24]:
get_autorank_df(datasets=["cora", "flickr", "ogbn-arxiv"])

Dataset             Cora             Flickr             OGBN-Arxiv        \
Model                GAT   GCN  SAGE    GAT   GCN  SAGE        GAT   GCN   
Similarity Measure                                                         
AlignCos           0.278 0.289 0.287  0.183 0.513 0.275      0.459 0.467   
CKA                0.274 0.273 0.274  0.141 0.445 0.415      0.728 0.846   
ConcDiff           0.212 0.131 0.198  0.156 0.218 0.235      0.505 0.518   
DistCorr           0.274 0.280 0.287  0.116 0.491 0.308      0.722 0.804   
EOS                0.132 0.160 0.168  0.123 0.268 0.284      0.222 0.170   
GULP               0.133 0.160 0.168  0.118 0.106 0.285      0.197 0.194   
HardCorr           0.270 0.272 0.273  0.191 0.389 0.332      0.334 0.520   
Jaccard            0.274 0.273 0.273  0.131 0.326 0.296      0.385 0.565   
LinReg             0.286 0.282 0.284  0.125 0.130 0.326      0.297 0.279   
MagDiff            0.185 0.119 0.134  0.110 0.410 0.273      0.172 0.270   
ShapeMet           0.273 0.273 0.276  0.143 0.537 0.313      0.454 0.494   
OrthProc           0.273 0.273 0.276  0.143 0.537 0.313      0.454 0.494   
PermProc           0.277 0.288 0.245  0.115 0.474 0.427      0.524 0.225   
ProcDist           0.280 0.286 0.290  0.115 0.456 0.394      0.624 0.425   
RSA                0.274 0.291 0.275  0.201 0.404 0.278      0.319 0.527   
RankSim            0.299 0.308 0.270  0.167 0.305 0.318      0.410 0.643   
SVCCA              0.272 0.223 0.193  0.137 0.237 0.408      0.658 0.869   
2nd-Cos            0.346 0.338 0.370  0.216 0.326 0.279      0.596 0.745   
SoftCorr           0.272 0.272 0.274  0.174 0.298 0.361      0.332 0.491   
UnifDiff           0.165 0.158 0.292  0.207 0.341 0.419      0.205 0.324   

Dataset                   Cora  ... OGBN-Arxiv   Cora               Flickr  \
Model               SAGE   GAT  ...       SAGE    GAT    GCN   SAGE    GAT   
Similarity Measure              ...                                          
AlignCos           0.464 1.000  ...      0.050 -0.369 -0.188  0.154  0.088   
CKA                0.800 1.000  ...     -0.021 -0.080 -0.310 -0.134 -0.095   
ConcDiff           0.543 0.403  ...      0.024 -0.098 -0.021 -0.213  0.078   
DistCorr           0.822 1.000  ...      0.122 -0.112 -0.299 -0.126 -0.041   
EOS                0.276 1.000  ...      0.115 -0.002  0.290  0.085  0.399   
GULP               0.277 1.000  ...      0.106 -0.037  0.245  0.085 -0.087   
HardCorr           0.515 0.540  ...     -0.276 -0.261  0.030 -0.116 -0.145   
Jaccard            0.283 1.000  ...      0.090 -0.226 -0.042 -0.100 -0.057   
LinReg             0.367 1.000  ...     -0.166 -0.205  0.009 -0.206 -0.115   
MagDiff            0.199 0.893  ...     -0.127 -0.073  0.051 -0.106  0.101   
ShapeMet           0.449 1.000  ...     -0.152 -0.237 -0.273 -0.263 -0.093   
OrthProc           0.449 1.000  ...     -0.152 -0.237 -0.273 -0.263 -0.093   
PermProc           0.257 1.000  ...     -0.552 -0.317  0.058  0.166  0.094   
ProcDist           0.480 1.000  ...     -0.503 -0.259 -0.047 -0.008  0.090   
RSA                0.284 1.000  ...      0.140 -0.331 -0.143  0.064  0.025   
RankSim            0.354 1.000  ...      0.358  0.035 -0.041 -0.093 -0.049   
SVCCA              0.723 0.613  ...      0.163 -0.122 -0.039 -0.098 -0.101   
2nd-Cos            0.710 1.000  ...      0.336 -0.207 -0.130  0.053 -0.000   
SoftCorr           0.531 0.500  ...     -0.284 -0.092 -0.130 -0.061 -0.200   
UnifDiff           0.244 0.590  ...      0.117  0.006 -0.125 -0.033  0.067   

Dataset                          OGBN-Arxiv                
Model                 GCN   SAGE        GAT    GCN   SAGE  
Similarity Measure                                         
AlignCos            0.173  0.388     -0.185 -0.069  0.201  
CKA                -0.150  0.372     -0.129  0.040  0.056  
ConcDiff           -0.110 -0.060     -0.191 -0.247 -0.004  
DistCorr            0.200  0.360     -0.127  0.138  0.010  
EOS                 0.227  0.224    

In [32]:
res_df = get_autorank_df().T.reset_index(drop=True)
result = autorank(res_df, alpha=0.05, verbose=False)
cd_diagram(result, False, None, PLOTS_CD_WIDTH)
fig = plt.gcf()
fig.set_size_inches(PLOTS_CD_WIDTH, PLOTS_CD_HEIGHT)
plt.savefig(f"cd_plot_graphs.png", bbox_inches="tight")
plt.close()

C:\Users\Tobias\anaconda3\envs\simba\lib\site-packages\autorank\_util.py:432: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if abs(sorted_ranks[i] - sorted_ranks[j]) <= critical_difference:
C:\Users\Tobias\anaconda3\envs\simba\lib\site-packages\autorank\_util.py:511: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  plot_line([(rankpos(sorted_ranks[i]), cline),
C:\Users\Tobias\anaconda3\envs\simba\lib\site-packages\autorank\_util.py:512: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To ac

In [29]:
get_autorank_df().T.reset_index(drop=True).shape

(54, 20)